In [1]:
import numpy as np

class RBM:
    def __init__(self, n_visible, n_hidden, learning_rate=0.1):
        """
        Initialize the RBM model.
        n_visible: Number of visible units (input nodes)
        n_hidden: Number of hidden units
        learning_rate: The learning rate for weight updates
        """
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights = np.random.randn(n_visible, n_hidden) * 0.01
        self.visible_bias = np.zeros(n_visible)
        self.hidden_bias = np.zeros(n_hidden)

    def sigmoid(self, x):
        """
        Sigmoid activation function
        """
        return 1.0 / (1 + np.exp(-x))

    def sample_hidden(self, visible):
        """
        Sample hidden units given the visible layer.
        """
        activation = np.dot(visible, self.weights) + self.hidden_bias
        probabilities = self.sigmoid(activation)
        return probabilities, np.random.binomial(1, probabilities)

    def sample_visible(self, hidden):
        """
        Sample visible units given the hidden layer.
        """
        activation = np.dot(hidden, self.weights.T) + self.visible_bias
        probabilities = self.sigmoid(activation)
        return probabilities, np.random.binomial(1, probabilities)

    def train(self, data, epochs=1000):
        """
        Train the RBM using Contrastive Divergence (CD-1).
        data: Training data (binary values, i.e., 0s and 1s)
        epochs: Number of training iterations
        """
        for epoch in range(epochs):
            # Positive phase: Calculate probabilities and sample hidden units
            positive_hidden_probs, positive_hidden_sample = self.sample_hidden(data)
            positive_associations = np.dot(data.T, positive_hidden_sample)

            # Negative phase: Reconstruct visible units from hidden samples
            negative_visible_probs, negative_visible_sample = self.sample_visible(positive_hidden_sample)
            negative_hidden_probs, _ = self.sample_hidden(negative_visible_sample)
            negative_associations = np.dot(negative_visible_sample.T, negative_hidden_probs)

            # Update weights and biases
            self.weights += self.learning_rate * (positive_associations - negative_associations) / data.shape[0]
            self.visible_bias += self.learning_rate * np.mean(data - negative_visible_sample, axis=0)
            self.hidden_bias += self.learning_rate * np.mean(positive_hidden_probs - negative_hidden_probs, axis=0)

            if epoch % 100 == 0:
                error = np.mean(np.abs(data - negative_visible_sample))
                print(f'Epoch {epoch}: Reconstruction error: {error}')

    def run_visible(self, visible):
        """
        Run visible units through the network to get hidden activations.
        """
        hidden_probs, hidden_sample = self.sample_hidden(visible)
        return hidden_sample

    def run_hidden(self, hidden):
        """
        Run hidden units through the network to get visible activations (reconstruction).
        """
        visible_probs, visible_sample = self.sample_visible(hidden)
        return visible_sample

# Example Usage
if __name__ == "__main__":
    # Define training data (e.g., binary data with 6 visible units)
    training_data = np.array([
        [1, 1, 1, 0, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 0]
    ])

    # Initialize RBM: 6 visible units, 2 hidden units
    rbm = RBM(n_visible=6, n_hidden=2)

    # Train RBM
    rbm.train(training_data, epochs=1000)

    # Example: Run input through the RBM and get hidden activations
    test_input = np.array([[1, 0, 1, 0, 0, 0]])  # Example test input
    hidden_representation = rbm.run_visible(test_input)
    print(f'Hidden representation for test input: {hidden_representation}')

    # Reconstruct the input from hidden units
    reconstruction = rbm.run_hidden(hidden_representation)
    print(f'Reconstruction from hidden representation: {reconstruction}')


Epoch 0: Reconstruction error: 0.3888888888888889
Epoch 100: Reconstruction error: 0.3611111111111111
Epoch 200: Reconstruction error: 0.3055555555555556
Epoch 300: Reconstruction error: 0.3888888888888889
Epoch 400: Reconstruction error: 0.16666666666666666
Epoch 500: Reconstruction error: 0.3055555555555556
Epoch 600: Reconstruction error: 0.25
Epoch 700: Reconstruction error: 0.2777777777777778
Epoch 800: Reconstruction error: 0.16666666666666666
Epoch 900: Reconstruction error: 0.2222222222222222
Hidden representation for test input: [[1 0]]
Reconstruction from hidden representation: [[0 0 1 0 0 0]]


Code Explanation
Initialization (__init__):

The RBM is initialized with the number of visible and hidden units.
Weights and biases are initialized. Weights are small random numbers, while biases start at zero.
Activation Function (sigmoid):

Sigmoid is used to compute the probability that a hidden (or visible) unit is activated, given its inputs.
Sampling Functions (sample_hidden, sample_visible):

These functions compute the activation probabilities of the hidden/visible units and then sample the activations using binomial sampling.
Training (train):

The RBM is trained using Contrastive Divergence (CD-1).
The algorithm performs a positive phase (sampling hidden units from data) and a negative phase (reconstructing visible units and resampling hidden units).
Weights and biases are updated using the difference between positive and negative associations.
Run Functions (run_visible, run_hidden):

These functions allow you to get the hidden representation from a visible input or reconstruct a visible input from hidden units.
Training Example:

A simple binary dataset is used to train the RBM. After training, the RBM can infer hidden representations and reconstruct visible data.

In [5]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler

# Define the training data (binary values)
training_data = np.array([
    [1, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 0]
])

# Normalize the data between 0 and 1 using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
training_data_scaled = scaler.fit_transform(training_data)

# Initialize the RBM
rbm = BernoulliRBM(n_components=2, learning_rate=0.1, n_iter=1000, random_state=42)

# Train the RBM
rbm.fit(training_data_scaled)

# Test example
test_input = np.array([[1, 0, 1, 0, 0, 0]])
test_input_scaled = scaler.transform(test_input)

# Transform visible input to hidden layer representation
hidden_representation = rbm.transform(test_input_scaled)
print(f'Hidden representation for test input: {hidden_representation}')

# Manually reconstruct visible layer from hidden layer
# reconstruction = sigmoid(np.dot(hidden_representation, rbm.components_) + rbm.intercept_visible_)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Manually perform the reconstruction
reconstruction_probabilities = sigmoid(np.dot(hidden_representation, rbm.components_) + rbm.intercept_visible_)
print(f'Reconstructed visible layer (probabilities): {reconstruction_probabilities}')

# Threshold the probabilities to get binary output (0 or 1)
reconstructed_input = (reconstruction_probabilities > 0.5).astype(int)
print(f'Reconstructed visible layer (binary): {reconstructed_input}')


Hidden representation for test input: [[0.01325487 0.01402122]]
Reconstructed visible layer (probabilities): [[0.29488918 0.19650378 0.00604107 0.29532034 0.18897966 0.0067882 ]]
Reconstructed visible layer (binary): [[0 0 0 0 0 0]]
